## 0.0 - IMPORTS

In [1]:
import pandas as pd
import numpy as np
import inflection

### 0.1 - Helper Functions

### 0.2 - Loading Data

In [2]:
df_train_raw = pd.read_csv('data/train.csv', low_memory=False)
df_store_raw = pd.read_csv('data/store.csv', low_memory=False)

# Merge dos dois Dataframes acima
df_raw = pd.merge(df_store_raw, df_train_raw, how='left', on='Store')

In [3]:
df_raw.head(1)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,5,2015-07-31,5263,555,1,1,0,1


## 1.0 - DATA DESCRIPTION

### 1.1 - Rename Columns

In [5]:
df1 = df_raw

In [16]:
cols_old = df1.columns

snakecase = lambda x : inflection.underscore(x)

cols_new = list(map(snakecase, cols_old))

# Rename
df1.columns = cols_new

In [17]:
df1.columns

Index(['store', 'store_type', 'assortment', 'competition_distance',
       'competition_open_since_month', 'competition_open_since_year', 'promo2',
       'promo2_since_week', 'promo2_since_year', 'promo_interval',
       'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday'],
      dtype='object')

### 1.2 - Data Dimensions

In [11]:
print(f'Number of rows: {df1.shape[0]}')
print(f'Number of columns: {df1.shape[1]}') 

Number of rows: 1017209
Number of columns: 18


### 1.3 - Data Types

In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   store                         1017209 non-null  int64  
 1   store_type                    1017209 non-null  object 
 2   assortment                    1017209 non-null  object 
 3   competition_distance          1014567 non-null  float64
 4   competition_open_since_month  693861 non-null   float64
 5   competition_open_since_year   693861 non-null   float64
 6   promo2                        1017209 non-null  int64  
 7   promo2_since_week             509178 non-null   float64
 8   promo2_since_year             509178 non-null   float64
 9   promo_interval                509178 non-null   object 
 10  day_of_week                   1017209 non-null  int64  
 11  date                          1017209 non-null  object 
 12  sales                       

In [13]:
df1.date = pd.to_datetime(df1.date) 

In [14]:
df1.dtypes

store                                    int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
dtype: object

### 1.4 - Check NA

In [15]:
df1.isna().sum()

store                                0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
dtype: int64

### 1.5

### 1.6

### 1.7